In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix,load_npz
import pickle
import tensorflow.compat.v1 as tf
from tensorflow.python.ops import confusion_matrix
from sklearn.metrics import mean_squared_error
tf.disable_v2_behavior()

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

--2021-10-03 13:30:58--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  15.6MB/s    in 0.3s    

2021-10-03 13:31:00 (15.6 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]



In [ ]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
trainData = pd.read_csv("ml-100k/u1.base",sep='\t',names=['user_id','item_id','rating','timestamp'])

In [ ]:
utilityMatrix=load_npz("utilityMatrix.npz")

In [ ]:
Ratings=utilityMatrix.toarray()

In [ ]:
mask = Ratings == 0
means = np.ma.array(Ratings, mask = mask).mean(0)
RatingsNonZero=Ratings + mask * means.data

In [ ]:
def getPredictedRatingsFromSVD1(Ratings,K=2,epochs=5000):
  U = np.random.rand(len(Ratings), K)
  M = np.random.rand(len(Ratings[0]), K)
  movie_features = tf.Variable(M, dtype=tf.float32)
  user_features = tf.Variable(U, dtype=tf.float32)
  ratingsPlaceholder = tf.placeholder(tf.float32,shape=(1682,943),name="ratings")
  Movie_Matmul_User = tf.matmul(movie_features, user_features, transpose_b=True)
  squaredError = tf.square(Movie_Matmul_User - ratingsPlaceholder)

  mse = tf.losses.mean_squared_error(Ratings.transpose(),Movie_Matmul_User)
  adam = tf.train.AdamOptimizer(learning_rate=0.3)
  train = adam.minimize(mse)
  sess = tf.Session()
  init = tf.global_variables_initializer()
  sess.run(init)
  for i in range(epochs):
      sess.run(train)
      print(f"Epoch: {i}, Loss: {sess.run(mse)}")

  final_movie_features = np.around(sess.run(movie_features), 3)
  final_user_features = np.around(sess.run(user_features), 3)
  predictedRatings = (np.matmul(final_movie_features,final_user_features.transpose()).transpose())
  return predictedRatings

In [ ]:
def getPredictedRatingsFromSVDRegularized(Ratings,K=2,epochs=5000,l1=0.001,l2=0.003):
  U = np.random.rand(len(Ratings), K)
  M = np.random.rand(len(Ratings[0]), K)
  movie_features = tf.Variable(M, dtype=tf.float32)
  user_features = tf.Variable(U, dtype=tf.float32)
  ratingsPlaceholder = tf.placeholder(tf.float32,shape=(1682,943),name="ratings")
  Movie_Matmul_User = tf.matmul(movie_features, user_features, transpose_b=True)
  squaredError = tf.square(Movie_Matmul_User - ratingsPlaceholder)

  mse = tf.losses.mean_squared_error(Ratings.transpose(),Movie_Matmul_User)+l1*tf.reduce_sum(tf.norm(movie_features)) + l2* tf.reduce_sum(tf.norm(user_features))
  adam = tf.train.AdamOptimizer(learning_rate=0.3)
  train = adam.minimize(mse)
  sess = tf.Session()
  init = tf.global_variables_initializer()
  sess.run(init)
  for i in range(epochs):
      sess.run(train)
      print(f"Epoch: {i}, Loss: {sess.run(mse)}")

  final_movie_features = np.around(sess.run(movie_features), 3)
  final_user_features = np.around(sess.run(user_features), 3)
  predictedRatings = (np.matmul(final_movie_features,final_user_features.transpose()).transpose())
  return predictedRatings

In [ ]:
def average_precision_k(dataset,k=10,threshold=3):
  # dataset['relevant_actual']=dataset['rating'].map(is_relevant)
  # dataset['relevant_predicted']=dataset['predicted_rating'].map(is_relevant)
  precisions=[]
  for user in range(943):
    dataframeUser = dataset[dataset['user_id']==user+1]
    dataframeUserActual = dataframeUser.sort_values(by='rating',ascending=False)
    dataframeUserPredicted = dataframeUser.sort_values(by='predicted_rating',ascending=False)
    actualReleventSet=set()
    predictedReleventSet=set()
    actualCounter=0
    predictedCounter=0
    for index,row in dataframeUserActual.iterrows():
      if(actualCounter==k):
        break
      if(row['rating']>=threshold):
        actualCounter+=1
        actualReleventSet.add(row['item_id'])
    for index,row in dataframeUserPredicted.iterrows():
      if(predictedCounter==k):
        break
      if(row['predicted_rating']>=threshold):
        predictedCounter+=1
        predictedReleventSet.add(row['item_id'])   
    if(len(actualReleventSet)==0):
      userPrecisionK==1
    else:
      userPrecisionK = len(actualReleventSet & predictedReleventSet)/len(actualReleventSet)   
    precisions.append(userPrecisionK)
  return sum(precisions)/len(precisions)

In [ ]:
predictedRatings=getPredictedRatingsFromSVD1(RatingsNonZero,K=2,epochs=5000)

Epoch: 0, Loss: 4.080177307128906
Epoch: 1, Loss: 1.3885530233383179
Epoch: 2, Loss: 0.9909466505050659
Epoch: 3, Loss: 1.7532726526260376
Epoch: 4, Loss: 1.5093498229980469
Epoch: 5, Loss: 0.7325727939605713
Epoch: 6, Loss: 0.34218132495880127
Epoch: 7, Loss: 0.4979003071784973
Epoch: 8, Loss: 0.8583574891090393
Epoch: 9, Loss: 1.0970276594161987
Epoch: 10, Loss: 1.0830445289611816
Epoch: 11, Loss: 0.8413013219833374
Epoch: 12, Loss: 0.49142584204673767
Epoch: 13, Loss: 0.19996409118175507
Epoch: 14, Loss: 0.11019095033407211
Epoch: 15, Loss: 0.2382080852985382
Epoch: 16, Loss: 0.4249779284000397
Epoch: 17, Loss: 0.4760280251502991
Epoch: 18, Loss: 0.3624851405620575
Epoch: 19, Loss: 0.21134130656719208
Epoch: 20, Loss: 0.14288170635700226
Epoch: 21, Loss: 0.17290838062763214
Epoch: 22, Loss: 0.23746903240680695
Epoch: 23, Loss: 0.26640784740448
Epoch: 24, Loss: 0.2329658418893814
Epoch: 25, Loss: 0.16121463477611542
Epoch: 26, Loss: 0.10252422839403152
Epoch: 27, Loss: 0.095635354518

In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.009269110858794

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7806826406773385

In [ ]:
predictedRatings=getPredictedRatingsFromSVD1(RatingsNonZero,K=10,epochs=5000)

Epoch: 0, Loss: 4.4812541007995605
Epoch: 1, Loss: 0.6542226672172546
Epoch: 2, Loss: 1.2195180654525757
Epoch: 3, Loss: 2.2075116634368896
Epoch: 4, Loss: 2.300959587097168
Epoch: 5, Loss: 1.6631619930267334
Epoch: 6, Loss: 0.9161055088043213
Epoch: 7, Loss: 0.607704222202301
Epoch: 8, Loss: 0.6995818614959717
Epoch: 9, Loss: 0.758937656879425
Epoch: 10, Loss: 0.6845119595527649
Epoch: 11, Loss: 0.6351431608200073
Epoch: 12, Loss: 0.6353436708450317
Epoch: 13, Loss: 0.597374677658081
Epoch: 14, Loss: 0.48723265528678894
Epoch: 15, Loss: 0.3709332048892975
Epoch: 16, Loss: 0.3302276134490967
Epoch: 17, Loss: 0.35195398330688477
Epoch: 18, Loss: 0.35273489356040955
Epoch: 19, Loss: 0.30379536747932434
Epoch: 20, Loss: 0.2518664300441742
Epoch: 21, Loss: 0.2344328612089157
Epoch: 22, Loss: 0.23885537683963776
Epoch: 23, Loss: 0.23347491025924683
Epoch: 24, Loss: 0.20476900041103363
Epoch: 25, Loss: 0.16970379650592804
Epoch: 26, Loss: 0.1529277265071869
Epoch: 27, Loss: 0.156147092580795

In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

0.9966057956213081

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7836725075325289

In [ ]:
predictedRatings=getPredictedRatingsFromSVD1(RatingsNonZero,K=5,epochs=5000)

Epoch: 0, Loss: 1.0081628561019897
Epoch: 1, Loss: 1.6746212244033813
Epoch: 2, Loss: 0.6836327910423279
Epoch: 3, Loss: 0.49654507637023926
Epoch: 4, Loss: 1.0477612018585205
Epoch: 5, Loss: 1.2294424772262573
Epoch: 6, Loss: 0.8638052940368652
Epoch: 7, Loss: 0.3330325484275818
Epoch: 8, Loss: 0.17243380844593048
Epoch: 9, Loss: 0.4644053280353546
Epoch: 10, Loss: 0.585597574710846
Epoch: 11, Loss: 0.3486568033695221
Epoch: 12, Loss: 0.1898425817489624
Epoch: 13, Loss: 0.272796630859375
Epoch: 14, Loss: 0.38603663444519043
Epoch: 15, Loss: 0.35226646065711975
Epoch: 16, Loss: 0.2148878425359726
Epoch: 17, Loss: 0.13735130429267883
Epoch: 18, Loss: 0.18360796570777893
Epoch: 19, Loss: 0.22753587365150452
Epoch: 20, Loss: 0.17480355501174927
Epoch: 21, Loss: 0.11556376516819
Epoch: 22, Loss: 0.134051114320755
Epoch: 23, Loss: 0.17703931033611298
Epoch: 24, Loss: 0.16512452065944672
Epoch: 25, Loss: 0.10751907527446747
Epoch: 26, Loss: 0.07973514497280121
Epoch: 27, Loss: 0.111360646784

In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0015938499450436

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0015938499450436

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.780983100203673

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=2,epochs=5000,l1=0.001,l2=0.003)

Epoch: 0, Loss: 4.302545547485352
Epoch: 1, Loss: 1.6588016748428345
Epoch: 2, Loss: 1.2439746856689453
Epoch: 3, Loss: 1.9285485744476318
Epoch: 4, Loss: 1.6901230812072754
Epoch: 5, Loss: 0.9587490558624268
Epoch: 6, Loss: 0.6025512218475342
Epoch: 7, Loss: 0.763231635093689
Epoch: 8, Loss: 1.1081132888793945
Epoch: 9, Loss: 1.3268258571624756
Epoch: 10, Loss: 1.2982354164123535
Epoch: 11, Loss: 1.0504454374313354
Epoch: 12, Loss: 0.703409731388092
Epoch: 13, Loss: 0.4226253628730774
Epoch: 14, Loss: 0.347459614276886
Epoch: 15, Loss: 0.4852445125579834
Epoch: 16, Loss: 0.6686191558837891
Epoch: 17, Loss: 0.7068110704421997
Epoch: 18, Loss: 0.58328777551651
Epoch: 19, Loss: 0.4312886595726013
Epoch: 20, Loss: 0.3678913712501526
Epoch: 21, Loss: 0.4027966260910034
Epoch: 22, Loss: 0.46882736682891846
Epoch: 23, Loss: 0.49652671813964844
Epoch: 24, Loss: 0.4618965983390808
Epoch: 25, Loss: 0.39124783873558044
Epoch: 26, Loss: 0.33541595935821533
Epoch: 27, Loss: 0.32950538396835327
Epo

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0146326594679989

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7782760187850327

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=10,epochs=5000,l1=0.001,l2=0.003)

Epoch: 0, Loss: 4.916569709777832
Epoch: 1, Loss: 0.9210167527198792
Epoch: 2, Loss: 1.5293495655059814
Epoch: 3, Loss: 2.5179970264434814
Epoch: 4, Loss: 2.6328787803649902
Epoch: 5, Loss: 2.031402587890625
Epoch: 6, Loss: 1.2755069732666016
Epoch: 7, Loss: 0.8999998569488525
Epoch: 8, Loss: 0.9232594966888428
Epoch: 9, Loss: 0.9758222103118896
Epoch: 10, Loss: 0.9269026517868042
Epoch: 11, Loss: 0.8954992890357971
Epoch: 12, Loss: 0.8990587592124939
Epoch: 13, Loss: 0.8603960275650024
Epoch: 14, Loss: 0.7534242272377014
Epoch: 15, Loss: 0.636867880821228
Epoch: 16, Loss: 0.5813321471214294
Epoch: 17, Loss: 0.5770981311798096
Epoch: 18, Loss: 0.5585992336273193
Epoch: 19, Loss: 0.5124304890632629
Epoch: 20, Loss: 0.48214060068130493
Epoch: 21, Loss: 0.4872177839279175
Epoch: 22, Loss: 0.4941420257091522
Epoch: 23, Loss: 0.4679124653339386
Epoch: 24, Loss: 0.42186465859413147
Epoch: 25, Loss: 0.3938336670398712
Epoch: 26, Loss: 0.38947778940200806
Epoch: 27, Loss: 0.3849230408668518
Ep

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0090323826478216

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7789741453315155

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=5,epochs=5000,l1=0.001,l2=0.003)

Epoch: 0, Loss: 1.2328565120697021
Epoch: 1, Loss: 1.8905128240585327
Epoch: 2, Loss: 0.9133496284484863
Epoch: 3, Loss: 0.7479271292686462
Epoch: 4, Loss: 1.2774516344070435
Epoch: 5, Loss: 1.4324951171875
Epoch: 6, Loss: 1.0626381635665894
Epoch: 7, Loss: 0.5565516948699951
Epoch: 8, Loss: 0.4304065704345703
Epoch: 9, Loss: 0.7141100764274597
Epoch: 10, Loss: 0.7911537885665894
Epoch: 11, Loss: 0.5441679954528809
Epoch: 12, Loss: 0.4081479609012604
Epoch: 13, Loss: 0.509859561920166
Epoch: 14, Loss: 0.622678279876709
Epoch: 15, Loss: 0.5782794952392578
Epoch: 16, Loss: 0.43552640080451965
Epoch: 17, Loss: 0.36087581515312195
Epoch: 18, Loss: 0.4059807360172272
Epoch: 19, Loss: 0.44244781136512756
Epoch: 20, Loss: 0.39324164390563965
Epoch: 21, Loss: 0.34903234243392944
Epoch: 22, Loss: 0.36965212225914
Epoch: 23, Loss: 0.3951268196105957
Epoch: 24, Loss: 0.36892616748809814
Epoch: 25, Loss: 0.32312968373298645
Epoch: 26, Loss: 0.3221273124217987
Epoch: 27, Loss: 0.3535146415233612
Ep

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0236711966512495

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.2597013920449764

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=2,epochs=5000,l1=0.001,l2=0.003)

Epoch: 0, Loss: 4.227213382720947
Epoch: 1, Loss: 1.60171377658844
Epoch: 2, Loss: 1.2343220710754395
Epoch: 3, Loss: 1.9429339170455933
Epoch: 4, Loss: 1.6882133483886719
Epoch: 5, Loss: 0.9423848986625671
Epoch: 6, Loss: 0.5888405442237854
Epoch: 7, Loss: 0.7573690414428711
Epoch: 8, Loss: 1.1057854890823364
Epoch: 9, Loss: 1.3229845762252808
Epoch: 10, Loss: 1.2905093431472778
Epoch: 11, Loss: 1.0394550561904907
Epoch: 12, Loss: 0.6923285722732544
Epoch: 13, Loss: 0.41586291790008545
Epoch: 14, Loss: 0.34767934679985046
Epoch: 15, Loss: 0.4898478090763092
Epoch: 16, Loss: 0.6707270741462708
Epoch: 17, Loss: 0.7024155855178833
Epoch: 18, Loss: 0.5752113461494446
Epoch: 19, Loss: 0.42511874437332153
Epoch: 20, Loss: 0.3664996922016144
Epoch: 21, Loss: 0.4050067067146301
Epoch: 22, Loss: 0.4715615510940552
Epoch: 23, Loss: 0.49729353189468384
Epoch: 24, Loss: 0.4599393606185913
Epoch: 25, Loss: 0.3875136375427246
Epoch: 26, Loss: 0.3318530321121216
Epoch: 27, Loss: 0.32785764336586
Epo

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0145639183992212

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7783820633237389

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=2,epochs=5000,l1=0.05,l2=0.05)

Epoch: 0, Loss: 8.385628700256348
Epoch: 1, Loss: 6.821666240692139
Epoch: 2, Loss: 6.653438568115234
Epoch: 3, Loss: 6.907112121582031
Epoch: 4, Loss: 6.781578063964844
Epoch: 5, Loss: 6.589334964752197
Epoch: 6, Loss: 6.536162853240967
Epoch: 7, Loss: 6.550361633300781
Epoch: 8, Loss: 6.535161018371582
Epoch: 9, Loss: 6.467785835266113
Epoch: 10, Loss: 6.380080223083496
Epoch: 11, Loss: 6.317911148071289
Epoch: 12, Loss: 6.303356647491455
Epoch: 13, Loss: 6.317020416259766
Epoch: 14, Loss: 6.318865776062012
Epoch: 15, Loss: 6.289910316467285
Epoch: 16, Loss: 6.246687889099121
Epoch: 17, Loss: 6.2152862548828125
Epoch: 18, Loss: 6.20477294921875
Epoch: 19, Loss: 6.205967903137207
Epoch: 20, Loss: 6.205095291137695
Epoch: 21, Loss: 6.195002555847168
Epoch: 22, Loss: 6.177870750427246
Epoch: 23, Loss: 6.161162376403809
Epoch: 24, Loss: 6.150873184204102
Epoch: 25, Loss: 6.1470818519592285
Epoch: 26, Loss: 6.145098686218262
Epoch: 27, Loss: 6.140781402587891
Epoch: 28, Loss: 6.1342172622

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.205417306030793

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.1930826979077243

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=5,epochs=5000,l1=0.05,l2=0.05)

Epoch: 0, Loss: 6.9726057052612305
Epoch: 1, Loss: 6.942602157592773
Epoch: 2, Loss: 6.706679821014404
Epoch: 3, Loss: 6.649468421936035
Epoch: 4, Loss: 6.609687805175781
Epoch: 5, Loss: 6.472101211547852
Epoch: 6, Loss: 6.349925994873047
Epoch: 7, Loss: 6.374570846557617
Epoch: 8, Loss: 6.457878589630127
Epoch: 9, Loss: 6.4199347496032715
Epoch: 10, Loss: 6.314202308654785
Epoch: 11, Loss: 6.270267486572266
Epoch: 12, Loss: 6.293769836425781
Epoch: 13, Loss: 6.304949760437012
Epoch: 14, Loss: 6.263357639312744
Epoch: 15, Loss: 6.1999664306640625
Epoch: 16, Loss: 6.167966842651367
Epoch: 17, Loss: 6.178315162658691
Epoch: 18, Loss: 6.190524578094482
Epoch: 19, Loss: 6.178377151489258
Epoch: 20, Loss: 6.160216331481934
Epoch: 21, Loss: 6.156739234924316
Epoch: 22, Loss: 6.1617279052734375
Epoch: 23, Loss: 6.159424781799316
Epoch: 24, Loss: 6.1475510597229
Epoch: 25, Loss: 6.135974884033203
Epoch: 26, Loss: 6.130690574645996
Epoch: 27, Loss: 6.127405166625977
Epoch: 28, Loss: 6.121827125

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.2053308348177634

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.19329478698513686

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=10,epochs=5000,l1=0.05,l2=0.05)

Epoch: 0, Loss: 7.949316024780273
Epoch: 1, Loss: 7.086727142333984
Epoch: 2, Loss: 6.831796646118164
Epoch: 3, Loss: 6.871707916259766
Epoch: 4, Loss: 6.698862075805664
Epoch: 5, Loss: 6.491351127624512
Epoch: 6, Loss: 6.465337753295898
Epoch: 7, Loss: 6.552041053771973
Epoch: 8, Loss: 6.5571489334106445
Epoch: 9, Loss: 6.473058700561523
Epoch: 10, Loss: 6.400740623474121
Epoch: 11, Loss: 6.369845390319824
Epoch: 12, Loss: 6.337352752685547
Epoch: 13, Loss: 6.291546821594238
Epoch: 14, Loss: 6.259131908416748
Epoch: 15, Loss: 6.243470191955566
Epoch: 16, Loss: 6.228036403656006
Epoch: 17, Loss: 6.21462345123291
Epoch: 18, Loss: 6.213214874267578
Epoch: 19, Loss: 6.217319488525391
Epoch: 20, Loss: 6.209992408752441
Epoch: 21, Loss: 6.187586784362793
Epoch: 22, Loss: 6.1661882400512695
Epoch: 23, Loss: 6.160042762756348
Epoch: 24, Loss: 6.161680221557617
Epoch: 25, Loss: 6.154452800750732
Epoch: 26, Loss: 6.139339447021484
Epoch: 27, Loss: 6.1312761306762695
Epoch: 28, Loss: 6.133584022

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.226692346424383

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.17948458987695465

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=2,epochs=5000,l1=0.50,l2=0.75)

Epoch: 0, Loss: 31.34380340576172
Epoch: 1, Loss: 20.52553367614746
Epoch: 2, Loss: 17.809751510620117
Epoch: 3, Loss: 20.426042556762695
Epoch: 4, Loss: 22.046127319335938
Epoch: 5, Loss: 21.431976318359375
Epoch: 6, Loss: 19.351486206054688
Epoch: 7, Loss: 17.023475646972656
Epoch: 8, Loss: 15.261659622192383
Epoch: 9, Loss: 14.825751304626465
Epoch: 10, Loss: 15.791182518005371
Epoch: 11, Loss: 16.525131225585938
Epoch: 12, Loss: 16.217878341674805
Epoch: 13, Loss: 14.986990928649902
Epoch: 14, Loss: 13.70728874206543
Epoch: 15, Loss: 13.296207427978516
Epoch: 16, Loss: 13.534496307373047
Epoch: 17, Loss: 13.946348190307617
Epoch: 18, Loss: 13.906728744506836
Epoch: 19, Loss: 13.325006484985352
Epoch: 20, Loss: 12.570000648498535
Epoch: 21, Loss: 12.272353172302246
Epoch: 22, Loss: 12.580602645874023
Epoch: 23, Loss: 12.789746284484863
Epoch: 24, Loss: 12.448166847229004
Epoch: 25, Loss: 11.952335357666016
Epoch: 26, Loss: 11.75263500213623
Epoch: 27, Loss: 11.934891700744629
Epoch:

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

3.7193435597080464

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.0

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=5,epochs=5000,l1=0.50,l2=0.75)

Epoch: 0, Loss: 43.446327209472656
Epoch: 1, Loss: 26.536273956298828
Epoch: 2, Loss: 21.817474365234375
Epoch: 3, Loss: 26.15799331665039
Epoch: 4, Loss: 28.801301956176758
Epoch: 5, Loss: 27.889541625976562
Epoch: 6, Loss: 24.64836883544922
Epoch: 7, Loss: 20.849552154541016
Epoch: 8, Loss: 17.911823272705078
Epoch: 9, Loss: 17.426538467407227
Epoch: 10, Loss: 19.138092041015625
Epoch: 11, Loss: 20.185590744018555
Epoch: 12, Loss: 19.573333740234375
Epoch: 13, Loss: 17.685874938964844
Epoch: 14, Loss: 15.719329833984375
Epoch: 15, Loss: 14.960498809814453
Epoch: 16, Loss: 15.53707218170166
Epoch: 17, Loss: 16.18122673034668
Epoch: 18, Loss: 16.010087966918945
Epoch: 19, Loss: 15.103151321411133
Epoch: 20, Loss: 13.922886848449707
Epoch: 21, Loss: 13.514592170715332
Epoch: 22, Loss: 14.056310653686523
Epoch: 23, Loss: 14.227974891662598
Epoch: 24, Loss: 13.763737678527832
Epoch: 25, Loss: 13.055501937866211
Epoch: 26, Loss: 12.68076229095459
Epoch: 27, Loss: 12.9823579788208
Epoch: 28

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

3.7194475573943477

In [ ]:
average_precision_k(testData,k=10,threshold=3)

In [ ]:
predictedRatings=getPredictedRatingsFromSVDRegularized(RatingsNonZero,K=10,epochs=5000,l1=0.50,l2=0.75)

Epoch: 0, Loss: 55.82781219482422
Epoch: 1, Loss: 32.42603302001953
Epoch: 2, Loss: 27.36508560180664
Epoch: 3, Loss: 32.714935302734375
Epoch: 4, Loss: 36.17292404174805
Epoch: 5, Loss: 35.65955352783203
Epoch: 6, Loss: 31.906524658203125
Epoch: 7, Loss: 26.309219360351562
Epoch: 8, Loss: 21.108123779296875
Epoch: 9, Loss: 20.020883560180664
Epoch: 10, Loss: 22.665468215942383
Epoch: 11, Loss: 24.31511688232422
Epoch: 12, Loss: 23.715482711791992
Epoch: 13, Loss: 21.522396087646484
Epoch: 14, Loss: 18.791048049926758
Epoch: 15, Loss: 16.860197067260742
Epoch: 16, Loss: 17.27433204650879
Epoch: 17, Loss: 18.568470001220703
Epoch: 18, Loss: 18.70156478881836
Epoch: 19, Loss: 17.5987548828125
Epoch: 20, Loss: 15.914405822753906
Epoch: 21, Loss: 14.846294403076172
Epoch: 22, Loss: 15.369077682495117
Epoch: 23, Loss: 15.942422866821289
Epoch: 24, Loss: 15.592277526855469
Epoch: 25, Loss: 14.581758499145508
Epoch: 26, Loss: 13.756895065307617
Epoch: 27, Loss: 13.98149585723877
Epoch: 28, Lo

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=predictedRatings[row['user_id']-1][row['item_id']-1]
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

3.7193405370341788

In [ ]:
average_precision_k(testData,k=10,threshold=3)